In [117]:
import threading
import time

In [ ]:
def get_board(state):
    board = [['X' for i in range(7)] for j in range(6)]
    for idx,chip in enumerate(state):
        row = idx//7
        col = idx%7
        board[abs(row-5)][col] = chip
    return board

def countSeries(board,chip,length):
    count = 0
    chip_length = length
    space_length = 4 - length
    
    for c in range(7):
        for r in range(6):
            
            #vertical
            if c < 7 and r >= 3 and r < 6: 
                if (board[r][c] == chip) and (board[r-1][c] == chip):
                    if length == 2 and (board[r-2][c] == 'X') and (board[r-3][c] == 'X'):
                        count += 1
                    if length == 3 and (board[r-2][c] == chip) and (board[r-3][c] == 'X'):
                        count += 1
                    if length == 4 and (board[r-2][c] == chip) and (board[r-3][c] == chip):
                        count += 1
                        
            #horizontal
            if c < 4 and r < 6:
                if (board[r][c] == chip) and (board[r][c+1] == chip):
                    if length == 2 and (board[r][c+2] == 'X') and (board[r][c+3] == 'X'):
                        count += 1
                    if length == 3 and (board[r][c+2] == chip) and (board[r][c+3] == 'X'):
                        count += 1
                    if length == 4 and (board[r][c+2] == chip) and (board[r][c+3] == chip):
                        count += 1
                        
            #positive diagonal
            if c < 4 and r >=3 and r < 6:
                if (board[r][c] == chip) and (board[r-1][c+1] == chip):
                    if length == 2 and (board[r-2][c+2] == 'X') and (board[r-3][c+3] == 'X'):
                        count += 1
                    if length == 3 and (board[r-2][c+2] == chip) and (board[r-3][c+3] == 'X'):
                        count += 1
                    if length == 4 and (board[r-2][c+2] == chip) and (board[r-3][c+3] == chip):
                        count += 1
                        
            #negative diagonal
            if c < 4 and r < 3:
                if (board[r][c] == chip) and (board[r+1][c+1] == chip):
                    if length == 2 and (board[r+2][c+2] == 'X') and (board[r+3][c+3] == 'X'):
                        count += 1
                    if length == 3 and (board[r+2][c+2] == chip) and (board[r+3][c+3] == 'X'):
                        count += 1
                    if length == 4 and (board[r+2][c+2] == chip) and (board[r+3][c+3] == chip):
                        count += 1
    return count
    

def is_win(board,chip):
    connect4 = countSeries(board,chip,4)
    if connect4 > 0:
        return True
    else:
        return False
    
def evaluate(board):
    p_connect2 = countSeries(board,'P',2)
    p_connect3 = countSeries(board,'P',3)
    p_connect4 = countSeries(board,'P',4)
    
    a_connect2 = countSeries(board,'A',2)
    a_connect3 = countSeries(board,'A',3)
    a_connect4 = countSeries(board,'A',4)
    
    score = (p_connect2-a_connect2)*10 + (p_connect3-a_connect3)*100 + (p_connect4-a_connect4)*1000
    
    if a_connect4 > 0:
        return float('-inf')
    else:
        return score
    
def is_valid_push(board,push_col):
    col = [row[push_col] for row in board]
    if 'X' in col:
        return True
    else:
        return False

def push(state,chip,push_col):
    board = get_board(state)
    for idx,pushed_chip in enumerate(state):
        if idx%7 == push_col and pushed_chip == 'X':
            state = state[:idx]+chip+state[idx+1:]
            return state

def max_value(state,a,b,level):
    board = get_board(state)
    
    if is_win(board,'P'):
        return -1000
    
    v = float('-inf')
    for push_col in range(7):
        if not is_valid_push(board,push_col):
            continue
        else:
            new_state = push(state,'A',push_col)
            v = max(v,min_value(new_state,a,b,level+1))
            if v >= b: 
                return v
            if a == -10:
                a = v
            else:
                a = max(a,v)
    return v

def min_value(state,a,b,level):
    board = get_board(state)
    
    if is_win(board,'A'):
        return 1000
    
    if level == max_depth:
        return evaluate(board)
    v = float('inf')
    for push_col in range(7):
        if not is_valid_push(board,push_col):
            continue
        else:
            new_state = push(state,'P',push_col)
            v = max(v,max_value(new_state,a,b,level+1))
            if v <= a: 
                return v
            if b == 10:
                b = v
            else:
                b = min(b,v)
    return v

def minimax_decision(state):
    global max_depth
    max_depth = 10
    max_value = float('-inf')
    a,b=-10,10
    min_score=[]
    count_win = 0
    count_lose = 0
    board = get_board(state)
    
    for push_col in range(7):
        if not is_valid_push(board,push_col):
            continue
        new_state = push(state,'P',push_col)
        ret = min_value(new_state,a,b,0)
        min_score.append(ret)
        if ret > max_value:
            max_value=ret
            action = push_col
        if ret == 1000:
            count_win += 1
        elif ret == -1000:
            count_lose += 1
    if count_win >= 1:
        print('From my calculation, I will win')
    elif count_lose == 7:
        print('From my calculation, you will win')
    print(min_score,action)
    return action

def count_down_thread():
    print('\n5..',end='')
    for i in range(4,-1,-1):
        time.sleep(1)
        if terminate_flag:
            break
        print(str(i)+'..',end='')
    print()
           

def show_board(state):
    board = get_board(state)
    for row in board:
        print(row+'/n')
    
is_ai_turn = False
state = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
show_board(state)
while 'X' in state:
    if is_ai_turn:
        c = int(input('Please enter your column:'))
        board = get_board(state)
        while not is_valid_push(board,c-1):
            print('You cannot put in column',c)
            c = int(input('Please enter your column:'))
        state = push(state,'A',c-1)
        board = get_board(state)
        if is_win(board,'A'):
            print('Perfext AI win!')
            show_board(state)
            break
        is_ai_turn = False
    else:
        ct=threading.Thread(target=count_down_thread)
        terminate_flag=False
        ct.start()
        c = minimax_decision(state)
        terminate_flag=True
        state = push(state,'P',c)
        board = get_board(state)
        show_board(board)
        if is_win(board,'P'):
            print('My minimax win!!!')
            breag
        is_ai_turn = True